In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
train = pd.read_csv('../Data/half_processed_train.csv')
test = pd.read_csv('../Data/processes_X_test.csv')

In [3]:
from sklearn.metrics import mean_squared_log_error, mean_absolute_error
from xgboost import XGBRegressor

In [4]:
X_train = train.drop(['xg_op_pred', 'rf_op_pred', 'time_spent'], axis=1)
y_train = train['time_spent']

In [5]:
from sklearn.model_selection import train_test_split
val_X_train, val_X_test, val_y_train, val_y_test = train_test_split(X_train, y_train, test_size=0.3)

In [6]:
# cols = ['session_number', 'purchased', 'added_in_cart', 'checked_out', 'app', 'iPhone', 'Android Tablet', 'Desktop', 'Other', 'Unknown', 'iPad']

cols = val_X_train.columns
len(cols)

30

In [7]:
op_log = []
training_loss = []

In [11]:
xg_r = XGBRegressor(n_estimators= 120, 
                    learning_rate= .01,
                    max_depth = 5, 
                    booster='gbtree', 
#                     min_child_weight = 1,
                    reg_alpha = 1000,
                    reg_lambda = 50)

xg_r.fit(val_X_train[cols], val_y_train,eval_metric='rmsle', eval_set=[(val_X_test[cols], val_y_test)], verbose=0)


if len(op_log) < 5 :
    op_log.append(np.sqrt(mean_squared_log_error(abs(xg_r.predict(val_X_test[cols])), val_y_test)))
else:
    op_log.pop(0)
    op_log.append(np.sqrt(mean_squared_log_error(abs(xg_r.predict(val_X_test[cols])), val_y_test)))
    
if len(training_loss) < 5 :
    training_loss.append(np.sqrt(mean_squared_log_error(abs(xg_r.predict(val_X_train[cols])), val_y_train)))
else:
    training_loss.pop(0)
    training_loss.append(np.sqrt(mean_squared_log_error(abs(xg_r.predict(val_X_train[cols])), val_y_train)))
    
op_df = pd.DataFrame(columns=['Training Loss', 'Validation Loss'])
op_df['Training Loss'] = np.array(training_loss)
op_df['Validation Loss'] = np.array(op_log)
op_df['Delta'] = abs(np.array(op_log) - np.array(training_loss))

print(op_df)

   Training Loss  Validation Loss     Delta
0       1.663705         1.656192  0.007514
1       1.663705         1.656192  0.007514
2       1.613032         1.634048  0.021016


In [9]:
print(pd.Series(xg_r.predict(val_X_test[cols])).nunique(),len(xg_r.predict(val_X_test[cols])))

193 1629


In [10]:
def save_submission(estimator, name):
    pred = estimator.predict(test[cols])
    sub = pd.read_csv('../Data/Sample Submission.csv')
    sub["time_spent"]=abs(pred)
    sub.to_csv('../Submissions/'+name+'.csv', index=False)

In [112]:
save_submission(xg_r, 'XGB_Manual_CV_'+str(op_log[-1]))

In [113]:
xg_r.save_model(fname='XGB_Manual_CV_'+str(op_log[-1]))